# Speed optimization and weather dataframes

In [ ]:
import os
import re
import json
import math
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import numpy as np
from natsort import index_natsorted

## General parameters

In [ ]:
PROJECT_PATH = os.path.dirname(os.path.abspath('.'))
NSO_PATH = f'{PROJECT_PATH}/output/solstorm/alns/managerial/speed_opt/nso/'
SO_PATH = f'{PROJECT_PATH}/output/solstorm/alns/managerial/speed_opt/so/'

DISC_PARAM = 4
WORST_WEATHER_STATE = 3
SPEED_IMPACTS = [0.0, 0.0, 2.0, 3.0]
SERVICE_IMPACTS = [1.0, 1.2, 1.3, 2.0]
DESIGN_SPEED = 12.0
FC_IDLING = 120
FC_SERVICING = 170

INSTANCE_KEY = 'instance'
NSO_P_KEY = 'nso (p)'
SO_P_KEY = 'so (p)'
NSO_M_KEY = 'nso (m)'
SO_M_KEY = 'so (m)'
NSO_C_KEY = 'nso (c)'
SO_C_KEY = 'so (c)'
RED_FC_P_KEY = 'cost red (p)'
RED_FC_M_KEY = 'cost red (m)'
RED_FC_C_KEY = 'cost red (c)'
RED_EM_P_KEY = 'em red (p)'
RED_EM_M_KEY = 'em red (m)'
RED_EM_C_KEY = 'em red (c)'

PERFECT_KEY = 'perfect'
MIXED_KEY = 'mixed'
CRITICAL_KEY = 'critical'

BEST_OBJ_KEY = 'best_objective'

VOYAGES_KEY = 'voyages'
SEQUENCE_KEY = 'sequence'
TIME_POINTS_KEY = 'time_points'
SPEED_KEY = 'speed'
ORDERS_KEY = 'orders'
INSTALLATION_KEY = 'installation'
LAT_KEY = 'latitude'
LON_KEY = 'longitude'
SCENARIOS_KEY = 'scenarios'
WS_KEY = 'weather_scenario'
END_DEPOT_KEY = 'ED'
ARR_TIME_KEY = 'arrival_time'
SERVICE_TIME_KEY = 'service_time'
END_TIME_KEY = 'end_time'
FLEET_KEY = 'fleet'
FC_DESIGN_SPEED_KEY = 'fc_design_speed'

## Functions

In [ ]:
def get_weather_forecast(weather_scenario):
    weather_path = f'{PROJECT_PATH}/src/main/resources/constant/weather.json'
    with open(weather_path) as file:
        weather_json = json.load(file)
    weather_forecast = weather_json[SCENARIOS_KEY][weather_scenario]
    wf = [weather_forecast[math.floor(i / DISC_PARAM)] for i in range(len(weather_forecast) * DISC_PARAM)]
    return wf

def get_latlon_list():
    installations_path = f'{PROJECT_PATH}/src/main/resources/constant/installations.json'
    with open(installations_path) as file:
        installations_json = json.load(file)
    latlon_list = []
    for inst in installations_json:
        latlon_list.append((installations_json[inst][LAT_KEY], installations_json[inst][LON_KEY]))
    return latlon_list

def get_instance_json(solution_json):
    instance_file = solution_json[INSTANCE_KEY]
    instance_path = f'{PROJECT_PATH}/src/main/resources/managerial_insights/{instance_file}'
    with open(instance_path) as file:
        instance_json = json.load(file)
    return instance_json

def get_vessels_json():
    vessels_path = f'{PROJECT_PATH}/src/main/resources/constant/vessels.json'
    with open(vessels_path) as file:
        vessels_json = json.load(file)
    return vessels_json

In [ ]:
def generate_speed_opt_df(nso_path, so_path):
    nso_obj_data = map_instance_to_ws_to_obj(nso_path)
    nso_em_data = map_instance_to_emissions(nso_path)
    so_obj_data = map_instance_to_ws_to_obj(so_path)
    so_em_data = map_instance_to_emissions(so_path)
    
    df = pd.DataFrame(columns=[INSTANCE_KEY, 
                               NSO_P_KEY, SO_P_KEY, RED_FC_P_KEY, RED_EM_P_KEY,
                               NSO_M_KEY, SO_M_KEY, RED_FC_M_KEY, RED_EM_M_KEY,
                               NSO_C_KEY, SO_C_KEY, RED_FC_C_KEY, RED_EM_C_KEY])

    for instance_name in nso_obj_data:
        perfect_obj_nso = nso_obj_data[instance_name][PERFECT_KEY]
        perfect_obj_so = so_obj_data[instance_name][PERFECT_KEY]
        perfect_fc_red = abs(perfect_obj_so - perfect_obj_nso) / perfect_obj_nso
        perfect_em_nso = nso_em_data[instance_name][PERFECT_KEY]
        perfect_em_so = so_em_data[instance_name][PERFECT_KEY]
        perfect_em_red = abs(perfect_em_so - perfect_em_nso) / perfect_em_nso
        
        mixed_obj_nso = nso_obj_data[instance_name][MIXED_KEY]
        mixed_obj_so = so_obj_data[instance_name][MIXED_KEY]
        mixed_fc_red = abs(mixed_obj_so - mixed_obj_nso) / mixed_obj_nso
        mixed_em_nso = nso_em_data[instance_name][MIXED_KEY]
        mixed_em_so = so_em_data[instance_name][MIXED_KEY]
        mixed_em_red = abs(mixed_em_so - mixed_em_nso) / mixed_em_nso
        
        critical_obj_nso = nso_obj_data[instance_name][CRITICAL_KEY]
        critical_obj_so = so_obj_data[instance_name][CRITICAL_KEY]
        critical_fc_red = abs(critical_obj_so - critical_obj_nso) / critical_obj_nso
        critical_em_nso = nso_em_data[instance_name][CRITICAL_KEY]
        critical_em_so = so_em_data[instance_name][CRITICAL_KEY]
        critical_em_red = abs(critical_em_so - critical_em_nso) / critical_em_nso
        
        row = pd.Series({INSTANCE_KEY: instance_name, 
                        NSO_P_KEY: perfect_obj_nso, SO_P_KEY: perfect_obj_so, 
                        RED_FC_P_KEY: perfect_fc_red, RED_EM_P_KEY: perfect_em_red,
                        NSO_M_KEY: mixed_obj_nso, SO_M_KEY: mixed_obj_so, 
                        RED_FC_M_KEY: mixed_fc_red, RED_EM_M_KEY: mixed_em_red,
                        NSO_C_KEY: critical_obj_nso, SO_C_KEY: critical_obj_so, 
                        RED_FC_C_KEY: critical_fc_red, RED_EM_C_KEY: critical_em_red})
        
        df = df.append(row, ignore_index=True)
    
    df = df.sort_values(by=INSTANCE_KEY,
                        key=lambda x: np.argsort(index_natsorted(df[INSTANCE_KEY])),
                        inplace=False)
    df = df.reset_index(drop=True)
    
    mean_row = pd.Series({INSTANCE_KEY: 'Mean values', 
                          NSO_P_KEY: df[NSO_P_KEY].mean(), SO_P_KEY: df[SO_P_KEY].mean(), 
                          RED_FC_P_KEY: df[RED_FC_P_KEY].mean(), RED_EM_P_KEY: df[RED_EM_P_KEY].mean(), 
                          NSO_M_KEY: df[NSO_M_KEY].mean(), SO_M_KEY: df[SO_M_KEY].mean(), 
                          RED_FC_M_KEY: df[RED_FC_M_KEY].mean(), RED_EM_M_KEY: df[RED_EM_M_KEY].mean(),
                          NSO_C_KEY: df[NSO_C_KEY].mean(), SO_C_KEY: df[SO_C_KEY].mean(), 
                          RED_FC_C_KEY: df[RED_FC_C_KEY].mean(), RED_EM_C_KEY: df[RED_EM_C_KEY].mean()})
    
    df = df.append(mean_row, ignore_index=True)
    df = df.round(4)
    return df
    
def map_instance_to_ws_to_obj(path):
    instance_to_ws_to_data = {}
    for file_name in os.listdir(path):
        if file_name == '.DS_Store':
            continue
        split_name = re.split('_|\.', file_name)
        instance_name = split_name[0]
        ws = split_name[1]
        solution_or_history = split_name[3]
        if solution_or_history == 'history':
            continue
        
        with open(path + file_name) as file:
            solution_json = json.load(file)
        
        obj = solution_json[BEST_OBJ_KEY]
        
        if instance_name in instance_to_ws_to_data:
            if ws in instance_to_ws_to_data[instance_name]:
                data = instance_to_ws_to_data[instance_name][ws]
                data[0] += obj
                data[1] += 1
            else:
                instance_to_ws_to_data[instance_name][ws] = [obj, 1]            
        else:
            instance_to_ws_to_data[instance_name] = {ws: [obj, 1]}
    
    instance_to_ws_to_obj = {}
    for instance_name in instance_to_ws_to_data:
        instance_to_ws_to_obj[instance_name] = {}
        for ws in instance_to_ws_to_data[instance_name]:
            data = instance_to_ws_to_data[instance_name][ws]
            instance_to_ws_to_obj[instance_name][ws] = data[0] / data[1]
            
    return instance_to_ws_to_obj

def map_instance_to_emissions(path):
    instance_to_ws_to_emissions_data = {}
    for file_name in os.listdir(path):
        if file_name == '.DS_Store':
            continue
        split_name = re.split('_|\.', file_name)
        instance_name = split_name[0]
        ws = split_name[1]
        
        solution_or_history = split_name[3]
        if solution_or_history == 'history':
            continue
        
        with open(path + file_name) as file:
            solution_json = json.load(file)
        
        emissions = calculate_emissions(solution_json)
        
        if instance_name in instance_to_ws_to_emissions_data:
            if ws in instance_to_ws_to_emissions_data[instance_name]:
                data = instance_to_ws_to_emissions_data[instance_name][ws]
                data[0] += emissions
                data[1] += 1
            else:
                instance_to_ws_to_emissions_data[instance_name][ws] = [emissions, 1]
        else:
            instance_to_ws_to_emissions_data[instance_name] = {ws: [emissions, 1]}
    
    instance_to_ws_to_emissions = {}
    for instance_name in instance_to_ws_to_emissions_data:
        instance_to_ws_to_emissions[instance_name] = {}
        for ws in instance_to_ws_to_emissions_data[instance_name]:
            data = instance_to_ws_to_emissions_data[instance_name][ws]
            instance_to_ws_to_emissions[instance_name][ws] = data[0] / data[1]
    
    return instance_to_ws_to_emissions
        

def calculate_emissions(solution_json):
    
    total_emissions = 0.0
    total_consumption = 0.0
    latlon_list = get_latlon_list()
    instance_json = get_instance_json(solution_json)
    vessels_json = get_vessels_json()
    weather_scenario = instance_json[WS_KEY]
    wf = get_weather_forecast(weather_scenario)
    orders_json = instance_json[ORDERS_KEY]
    
    for psv in solution_json[VOYAGES_KEY]:
        
        fc_design_speed = vessels_json[FLEET_KEY][psv][FC_DESIGN_SPEED_KEY]
        
        sail_consumption = 0.0
        idle_consumption = 0.0
        service_consumption = 0.0
        
        sequence = solution_json[VOYAGES_KEY][psv][SEQUENCE_KEY]
        if len(sequence) == 0:
            continue
        
        from_inst = 0
        prev_end_time = 63
        
        time_points_json = solution_json[VOYAGES_KEY][psv][TIME_POINTS_KEY]

        first_order_idx = sequence[0]
        depot_inst_idx = 0
        first_order_inst_idx = orders_json[str(first_order_idx)][INSTALLATION_KEY]
        start_time = 63
        arr_time = time_points_json[str(first_order_idx)][ARR_TIME_KEY]
        service_time = time_points_json[str(first_order_idx)][SERVICE_TIME_KEY]
        end_time = time_points_json[str(first_order_idx)][END_TIME_KEY]
        speed = time_points_json[str(first_order_idx)][SPEED_KEY]
        depot_inst_latlon = latlon_list[depot_inst_idx]
        first_order_inst_latlon = latlon_list[first_order_inst_idx]
        dist = calculate_distance(depot_inst_latlon[0], first_order_inst_latlon[0], 
                                  depot_inst_latlon[1], first_order_inst_latlon[1])
        sail_consumption += calculate_total_fc_sailing(start_time, arr_time, speed, dist, fc_design_speed, wf)
        idle_consumption += calculate_fc_idling(arr_time, service_time, wf)
        service_consumption += calculate_fc_servicing(service_time, end_time, wf)

        for i in range(0, len(sequence) - 1):
            from_order_idx = sequence[i]
            to_order_idx = sequence[i + 1]
            from_order_inst_idx = orders_json[str(from_order_idx)][INSTALLATION_KEY]
            to_order_inst_idx = orders_json[str(to_order_idx)][INSTALLATION_KEY]
            start_time = time_points_json[str(from_order_idx)][END_TIME_KEY]
            arr_time = time_points_json[str(to_order_idx)][ARR_TIME_KEY]
            service_time = time_points_json[str(to_order_idx)][SERVICE_TIME_KEY]
            end_time = time_points_json[str(to_order_idx)][END_TIME_KEY]
            speed = time_points_json[str(to_order_idx)][SPEED_KEY]
            from_order_inst_latlon = latlon_list[from_order_inst_idx]
            to_order_inst_latlon = latlon_list[to_order_inst_idx]
            dist = calculate_distance(from_order_inst_latlon[0], to_order_inst_latlon[0],
                                      from_order_inst_latlon[1], to_order_inst_latlon[1])
            sail_consumption += calculate_total_fc_sailing(start_time, arr_time, speed, dist, fc_design_speed, wf)
            idle_consumption += calculate_fc_idling(arr_time, service_time, wf)
            service_consumption += calculate_fc_servicing(service_time, end_time, wf)
        
        last_order_idx = sequence[-1]
        last_order_inst_idx = orders_json[str(last_order_idx)][INSTALLATION_KEY]
        start_time = time_points_json[str(last_order_idx)][END_TIME_KEY]
        arr_time = time_points_json[END_DEPOT_KEY][END_TIME_KEY]
        speed = time_points_json[END_DEPOT_KEY][SPEED_KEY]
        last_order_inst_latlon = latlon_list[last_order_inst_idx]
        dist = calculate_distance(last_order_inst_latlon[0], depot_inst_latlon[0],
                                  last_order_inst_latlon[1], depot_inst_latlon[1])
        sail_consumption += calculate_total_fc_sailing(start_time, arr_time, speed, dist, fc_design_speed, wf)
        
        total_consumption += sail_consumption + idle_consumption + service_consumption
        total_emissions += (sail_consumption + idle_consumption + service_consumption) * 3
        
        # to_inst = 0

        # speed = time_points_json[END_DEPOT_KEY][SPEED_KEY]
        
        # arr_time = time_points_json[END_DEPOT_KEY][ARR_TIME_KEY]
        # service_time = time_points_json[str(order)][SERVICE_TIME_KEY]
        # end_time = time_points_json[str(order)][END_TIME_KEY]
        
        # from_inst_latlon = latlon_list[from_inst]
        # to_inst_latlon = latlon_list[to_inst]
        # distance = calculate_distance(from_inst_latlon[0], to_inst_latlon[0], from_inst_latlon[1], to_inst_latlon[1])
        
        # fc_sailing = calculate_total_fc_sailing(prev_end_time, arr_time, speed, distance, fc_design_speed, wf)
        # em_sailing = fc_sailing * 3
        
        # fc_idling = calculate_fc_idling(arr_time, service_time, wf)
        # em_idling = fc_idling * 3
        
        # fc_servicing = calculate_fc_servicing(service_time, end_time, wf)
        # em_servicing = fc_servicing * 3
        
        # total_emissions += em_sailing + em_idling + em_servicing
    
    return total_emissions

def calculate_distance(lat1, lat2, lon1, lon2):
    if lat1 == lat2 and lon1 == lon2:
        return 0

    theta = lon1 - lon2
    dist = math.sin(math.radians(lat1)) * math.sin(math.radians(lat2)) \
           + math.cos(math.radians(lat1)) * math.cos(math.radians(lat2)) * math.cos(math.radians(theta))

    dist = math.acos(dist)
    dist = math.degrees(dist)
    dist = dist * 60 * 1.1515
    dist = dist * 0.8684  # Nautical miles
    return dist

def calculate_total_fc_sailing(start_time, arr_time, speed, distance, fc_design_speed, wf):
    if distance == 0 or start_time == arr_time:
        return 0
    time_in_each_ws = get_time_in_each_weather_state(start_time, arr_time, wf)
    distance_in_each_ws = [speed * time_in_each_ws[ws] for ws in range(WORST_WEATHER_STATE + 1)]
    consumption = calculate_fc_sailing(distance_in_each_ws[0] + distance_in_each_ws[1], 
                                       speed, 0, fc_design_speed) \
                  + calculate_fc_sailing(distance_in_each_ws[2], speed, 2, fc_design_speed) \
                  + calculate_fc_sailing(distance_in_each_ws[3], speed, 3, fc_design_speed)
    return consumption

def calculate_fc_sailing(distance, speed, weather, fc_design_speed):
    return (distance / (speed - SPEED_IMPACTS[weather])) * fc_design_speed * math.pow((speed / DESIGN_SPEED), 3)

def calculate_fc_idling(idling_start_time, idling_end_time, wf):
    time_in_each_ws = get_time_in_each_weather_state(idling_start_time, idling_end_time, wf)
    consumption = 0
    for ws in range(WORST_WEATHER_STATE + 1):
        consumption += time_in_each_ws[ws] * SERVICE_IMPACTS[ws] * FC_IDLING
    return consumption

def calculate_fc_servicing(servicing_start_time, servicing_end_time, wf):
    time_in_each_ws = get_time_in_each_weather_state(servicing_start_time, servicing_end_time, wf)
    consumption = 0
    for ws in range(WORST_WEATHER_STATE + 1):
        consumption += time_in_each_ws[ws] * SERVICE_IMPACTS[ws] * SERVICE_IMPACTS[ws] * FC_SERVICING
    return consumption

def calculate_service_time(order_size):
    if order_size == 0:  # For pickup
        return 1
    else:
        return math.ceil(order_size * 0.4)

def get_time_in_each_weather_state(start_time, end_time, wf):
    return [disc_to_exact_hours(get_time_in_weather_state(start_time, end_time, ws, wf))
            for ws in range(WORST_WEATHER_STATE + 1)]

def get_time_in_weather_state(start_time, end_time, weather_state, wf):
    curr_time = start_time
    time_spent_in_weather_state = 0
    while curr_time < end_time:
        if weather_state == wf[curr_time]:
            time_spent_in_weather_state += 1
        curr_time += 1
    return time_spent_in_weather_state

def disc_to_exact_hours(disc_time):
    return disc_time / DISC_PARAM

def hour_to_disc(hourly_time):
    return hourly_time * DISC_PARAM

def deduce_postponed_orders(solution_path):
    with open(solution_path) as file:
        solution_json = json.load(file)
    
    instance_file = solution_json[INSTANCE_KEY]
    instance_path = f'{PROJECT_PATH}/src/main/resources/managerial_insights/{instance_file}'
    with open(instance_path) as file:
        instance_json = json.load(file)
    
    all_orders = []
    for order in instance_json[ORDERS_KEY]:
        all_orders.append(int(order))
    
    for psv in solution_json[VOYAGES_KEY]:
        sequence = solution_json[VOYAGES_KEY][psv][SEQUENCE_KEY]
        for order in sequence:
            if order in all_orders:
                all_orders.remove(order)
                
    print('Postponed orders: ' + str(all_orders))
    
def find_storm_periods(weather_forecast):
    for time_point, ws in enumerate(weather_forecast):
        if ws == WORST_WEATHER_STATE:
            print(time_point, ws)
            
def print_sequences(solution_path):
    with open(solution_path) as file:
        solution_json = json.load(file)
    
    instance_file = solution_json[INSTANCE_KEY]
    instance_path = f'{PROJECT_PATH}/src/main/resources/managerial_insights/{instance_file}'
    with open(instance_path) as file:
        instance_json = json.load(file)
    
    orders_json = instance_json[ORDERS_KEY]
    
    for psv in solution_json[VOYAGES_KEY]:
        print(psv)
        sequence = solution_json[VOYAGES_KEY][psv][SEQUENCE_KEY]
        from_inst = 0
        prev_end_time = 63
        for order in sequence:
            to_inst = orders_json[str(order)][INSTALLATION_KEY]
            time_points_json = solution_json[VOYAGES_KEY][psv][TIME_POINTS_KEY]
            if not bool(time_points_json):
                continue
            arr_time = time_points_json[str(order)][ARR_TIME_KEY]
            service_time = time_points_json[str(order)][SERVICE_TIME_KEY]
            end_time = time_points_json[str(order)][END_TIME_KEY]

            print(f'  Order {order}: ({from_inst}, {prev_end_time}) -> ({to_inst}, {end_time})')

            prev_end_time = end_time
            from_inst = to_inst
        print()
    
def calculate_penalty_cost(opt_order_size, inst_id, weather_scenario):  # 0 if pickup!
    wf = get_weather_forecast(weather_scenario)
    
    installations_path = f'{PROJECT_PATH}/src/main/resources/constant/installations.json'
    with open(installations_path) as file:
        installations_json = json.load(file)
    
    latlon_list = []
    for inst in installations_json:
        latlon_list.append((installations_json[inst][LAT_KEY], installations_json[inst][LON_KEY]))
    
    latlon_depot = latlon_list[0]
    latlon_inst = latlon_list[inst_id]
    distance = calculate_distance(latlon_depot[0], latlon_inst[0], latlon_depot[1], latlon_inst[1])
    
    start_time = 63
    arr_time = start_time + math.floor(hour_to_disc(distance / DESIGN_SPEED))
    service_end_time = arr_time + calculate_service_time(opt_order_size)
    sail_cost = (calculate_total_fc_sailing(start_time, arr_time, DESIGN_SPEED, distance, 540, wf) * 0.276) * 2
    service_cost = calculate_fc_servicing(arr_time, service_end_time, wf) * 0.276

    return sail_cost + service_cost
    
def calculate_idling_cost(start_time, end_time, weather_scenario):
    wf = get_weather_forecast(weather_scenario)
    return calculate_fc_idling(start_time, end_time, wf) * 0.276

In [ ]:
# with open(f"{NSO_PATH}/23-27-4-1_critical_135_solution.json") as file:
#     nso_json = json.load(file)
# calculate_emissions(nso_json)    

with open(f"{SO_PATH}/23-27-4-1_critical_917_solution.json") as file:
    so_json = json.load(file)
calculate_emissions(so_json)

## Speed optimization dataframe

In [ ]:
speedopt_df = generate_speed_opt_df(NSO_PATH, SO_PATH)
speedopt_df

## Weather dataframe

In [ ]:
FILE_NAME = '19-21-3-2_perfect_772_solution.json'
FILE_PATH = f'{PROJECT_PATH}/output/solstorm/alns/managerial/weather/{FILE_NAME}'
deduce_postponed_orders(FILE_PATH)

In [ ]:
print_sequences(FILE_PATH)

In [ ]:
calculate_penalty_cost(0, 2, 'critical')

In [ ]:
calculate_idling_cost(152, 320, 'critical')

In [ ]:
WEATHER_SCENARIO = 'critical'
weather_path = f'{PROJECT_PATH}/src/main/resources/constant/weather.json'
with open(weather_path) as file:
    weather_json = json.load(file)
weather_forecast = weather_json[SCENARIOS_KEY][WEATHER_SCENARIO]
wf = [weather_forecast[math.floor(i / DISC_PARAM)] for i in range(len(weather_forecast) * DISC_PARAM)]
find_storm_periods(wf)